# Evaluate

This notebook evaluates the network trained previous notebooks and analyzes the results.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%cd ..
%load_ext autoreload
%autoreload 2

C:\Users\Minjae\Desktop\EEG_Project


In [2]:
# Load some packages
import os
import sys
import pickle
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
from collections import OrderedDict

import numpy as np
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

import pprint
import wandb
import matplotlib
import matplotlib.pyplot as plt

# custom package
from datasets.caueeg_script import build_dataset_for_train
import models
from train.evaluate import check_accuracy
from train.evaluate import check_accuracy_extended
from train.evaluate import check_accuracy_extended_debug
from train.evaluate import check_accuracy_multicrop
from train.evaluate import check_accuracy_multicrop_extended
from train.evaluate import calculate_class_wise_metrics
from train.visualize import draw_roc_curve
from train.visualize import draw_confusion
from train.visualize import draw_class_wise_metrics
from train.visualize import draw_error_table
from train.visualize import annotate_heatmap

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.11.0+cu113
cuda is available.


-----

## List up the models to check accuracy

In [1]:
model_names = [
'lo88puq7',
'glem9euv',
'ehh4f6gj',
'1sl7ipca',
'gefkh07l',
'v301o425',
'l8524nml',
't1xzqrjh',
'ikik8fuj',
'gvqyvmrj',
'j5ybdq18',
'gchbqf3f',
'gexwncpq',
'2svr4k2h',
'vuk1myed',
'cix5o8ut',
'cgszsx9e',
'i8bx2s6r',
'xci5svkl',
'6a1tnz3r',
'r7fr150u',
'1tdyketc',
'4lv906r9',
'gjkysllw',
't0j6ipeb',
'd37sa3wb',
'z6yuten2',
]

model_pool = []

for model_name in model_names:
    path = os.path.join(r'E:\CAUEEG\checkpoint', model_name, 'checkpoint.pt')
    try:
        ckpt = torch.load(path, map_location=device)
        model_pool.append({'name': model_name, 'path': path})
    except Exception as e:
        print(e)
        print(f'- checkpoint cannot be opened: {path}')
        
pprint.pprint([model_dict['name'] for model_dict in model_pool])

NameError: name 'os' is not defined

In [5]:
# model_pool = []

# api = wandb.Api()
# runs = api.runs('ipis-mjkim/caueeg-task2-ablation')

# for run in runs:
#     path = os.path.join(r'E:\CAUEEG\checkpoint', run.name, 'checkpoint.pt')
#     try:
#         ckpt = torch.load(path, map_location=device)
#         model_pool.append({'name': run.name, 'path': path})
#     except Exception as e:
#         print(e)
#         print(f'- {run.name}\'s checkpoint cannot be opened: {path}')
        
# pprint.pprint([model_dict['name'] for model_dict in model_pool])

---

## Configurations

In [6]:
base_repeat = 8  # 200
verbose = False

-----

## Load and check accuracy

In [7]:
for model_dict in model_pool:
    # load and parse the checkpoint
    ckpt = torch.load(model_dict['path'], map_location=device)
    model_state = ckpt['model_state']
    config = ckpt['config']
    
    model_dict['model'] = config['model']
    model_dict['num_params'] = config.get('num_params', '???')
    model_dict['model size (MiB)'] = sys.getsizeof(pickle.dumps(model_state)) / (1024 * 1024)
    # torch.save(model_state, 'temptemptemp.pt')
    
    model_dict['seq_length'] = config['seq_length']
    model_dict['use_age'] = config['use_age']
    model_dict['photic'] = config['photic']
    model_dict['EKG'] = config['EKG']

    model_dict['awgn'] = config.get('awgn', 0)
    model_dict['awgn_age'] = config.get('awgn_age', 0)
    model_dict['mgn'] = config.get('mgn', 0)
    model_dict['mixup'] = config.get('mixup', 0)
    model_dict['dropout'] = config.get('dropout', 0)
    model_dict['weight_decay'] = config.get('weight_decay', '???')
    model_dict['fc_stages'] = config.get('fc_stages', 1)
    
    model_dict['minibatch'] = round(config['minibatch'])
    model_dict['total_samples'] = round(config.get('total_samples', config['iterations'] * config['minibatch']))
    model_dict['base_lr'] = config.get('base_lr', config.get('LR', '???'))
    model_dict['lr_scheduler_type'] = config.get('lr_scheduler_type', 'constant_with_decay')
    model_dict['warmup_steps'] = config.get('warmup_steps', '???')
    model_dict['seed'] = config.get('seed', '???')
    
    print('- checking for', model_dict['name'], config['model'], '...')
    
    # initiate the model
    if '_target_' in config:
        model = hydra.utils.instantiate(config).to(device)
    elif type(config['generator']) is str:
        config['generator'] = getattr(models, config['generator'].split('.')[-1])
        if 'block' in config:
            config['block'] = getattr(models, config['block'].split('.')[-1])
        model = config['generator'](**config).to(device)
    else:
        if 'block' in config:
            if config['block'] == models.resnet_1d.BottleneckBlock1D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_2d.Bottleneck2D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_1d.BasicBlock1D:
                config['block'] = 'basic'
            elif config['block'] == models.resnet_2d.BasicBlock2D:
                config['block'] = 'basic'
                
        model = config['generator'](**config).to(device)
    
    if config.get('ddp', False):
        model_state_ddp = deepcopy(model_state)
        model_state = OrderedDict()
        for k, v in model_state_ddp.items():
            name = k[7:]  # remove 'module.' of DataParallel/DistributedDataParallel
            model_state[name] = v
    
    model.load_state_dict(model_state)
    
    # reconfigure and update
    config.pop('cwd', 0)
    config['ddp'] = False
    config['crop_timing_analysis'] = False
    config['eval'] = True
    config['device'] = device
    
    repeat = round(base_repeat / config['crop_multiple'])
    model_dict['repeat'] = repeat
    model_dict['crop_multiple'] = config['crop_multiple']
    model_dict['test_crop_multiple'] = config['test_crop_multiple']
    
    # build dataset
    _ = build_dataset_for_train(config, verbose=verbose)
    train_loader = _[0]
    val_loader = _[1]
    test_loader = _[2]
    multicrop_test_loader = _[3]
    
    # train accuracy
    train_acc = check_accuracy(model, train_loader, 
                               config['preprocess_test'], config, repeat=repeat)
    model_dict['Train Accuracy'] = train_acc
    
    # val accuracy
    val_acc = check_accuracy(model, val_loader, 
                             config['preprocess_test'], config, repeat=repeat)
    model_dict['Validation Accuracy'] = val_acc
    
    # Test accuracy
    _ = check_accuracy_extended(model, test_loader, 
                                config['preprocess_test'], config, repeat=repeat)
    model_dict['Test Accuracy'] = _[0]
    model_dict['Test Throughput'] = _[4]
    test_class_wise_metrics = calculate_class_wise_metrics(_[3])
    
    for k, v in test_class_wise_metrics.items():
        for c in range(config['out_dims']):
            c_name = config['class_label_to_name'][c]
            model_dict[f'{k} ({c_name})'] = test_class_wise_metrics[k][c]
    
    # Multi-crop test accuracy
    _ = check_accuracy_multicrop_extended(model, multicrop_test_loader, 
                                          config['preprocess_test'], config, repeat=repeat)
    model_dict['Multi-Crop Test Accuracy'] = _[0]
    model_dict['Multi-Crop Test Throughput'] = _[4]
    multi_test_class_wise_metrics = calculate_class_wise_metrics(_[3])
    
    for k, v in multi_test_class_wise_metrics.items():
        for c in range(config['out_dims']):
            c_name = config['class_label_to_name'][c]
            model_dict[f'Multi-Crop {k} ({c_name})'] = multi_test_class_wise_metrics[k][c]
            
print('==== Finished ====')

- checking for lo88puq7 2D-VGG-19 ...
- checking for glem9euv 2D-VGG-19 ...
- checking for ehh4f6gj 1D-ResNet-18 ...
- checking for 1sl7ipca 2D-ResNeXt-50 ...
==== Finished ====


In [8]:
pprint.pprint(model_pool)

[{'Accuracy (Dementia)': 0.809322033898305,
  'Accuracy (MCI)': 0.6875,
  'Accuracy (Normal)': 0.8358050847457628,
  'EKG': 'O',
  'F1-score (Dementia)': 0.5477386934673367,
  'F1-score (MCI)': 0.565537555228277,
  'F1-score (Normal)': 0.8088779284833538,
  'Multi-Crop Accuracy (Dementia)': 0.826271186440678,
  'Multi-Crop Accuracy (MCI)': 0.7372881355932204,
  'Multi-Crop Accuracy (Normal)': 0.8432203389830508,
  'Multi-Crop F1-score (Dementia)': 0.5858585858585857,
  'Multi-Crop F1-score (MCI)': 0.6352941176470588,
  'Multi-Crop F1-score (Normal)': 0.8177339901477833,
  'Multi-Crop Precision (Dementia)': 0.7837837837837838,
  'Multi-Crop Precision (MCI)': 0.6136363636363636,
  'Multi-Crop Precision (Normal)': 0.7477477477477478,
  'Multi-Crop Recall (Dementia)': 0.46774193548387094,
  'Multi-Crop Recall (MCI)': 0.6585365853658537,
  'Multi-Crop Recall (Normal)': 0.9021739130434783,
  'Multi-Crop Sensitivity (Dementia)': 0.46774193548387094,
  'Multi-Crop Sensitivity (MCI)': 0.6585365

In [9]:
pd.DataFrame(model_pool)

,name,path,model,num_params,model size (MiB),seq_length,use_age,photic,EKG,awgn,...,Multi-Crop Specificity (Dementia),Multi-Crop Precision (Normal),Multi-Crop Precision (MCI),Multi-Crop Precision (Dementia),Multi-Crop Recall (Normal),Multi-Crop Recall (MCI),Multi-Crop Recall (Dementia),Multi-Crop F1-score (Normal),Multi-Crop F1-score (MCI),Multi-Crop F1-score (Dementia)
0,lo88puq7,E:\CAUEEG\checkpoint\lo88puq7\checkpoint.pt,2D-VGG-19,20184131,77.073749,4000,conv,X,O,0.047940,...,0.954023,0.747748,0.613636,0.783784,0.902174,0.658537,0.467742,0.817734,0.635294,0.585859
1,glem9euv,E:\CAUEEG\checkpoint\glem9euv\checkpoint.pt,2D-VGG-19,20225763,77.239990,1000,conv,X,X,0.045852,...,0.936782,0.722689,0.586667,0.738095,0.934783,0.536585,0.500000,0.815166,0.560510,0.596154
2,ehh4f6gj,E:\CAUEEG\checkpoint\ehh4f6gj\checkpoint.pt,1D-ResNet-18,11358787,43.413528,4000,conv,X,O,0.078195,...,0.862069,0.766990,0.565217,0.625000,0.858696,0.475610,0.645161,0.810256,0.516556,0.634921
3,1sl7ipca,E:\CAUEEG\checkpoint\1sl7ipca\checkpoint.pt,2D-ResNeXt-50,25886467,99.136700,4000,fc,O,X,0.103950,...,0.931034,0.708333,0.605634,0.733333,0.923913,0.524390,0.532258,0.801887,0.562092,0.616822


In [10]:
pd.DataFrame(model_pool).to_csv('local/output/caueeg-task2-summary.csv')